In [4]:
import uproot

the file here is a part of `mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026`:
```
/data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1
```

In [2]:
! ls -lh /data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1

-rw-r--r-- 1 alheld atlas-af 530M Mar 29 13:07 /data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1


In [16]:
fname = "/data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1"
t = uproot.open({fname: "CollectionTree"})

size_per_key_in_MB = {}
cur_size_in_MB = t.file.source.num_requested_bytes/1000**2  # non-zero at the start (some metadata read)

for key in t.keys():
    # if not any([key.startswith(v) for v in ["AnalysisJetsAuxDyn", "AnalysisLargeRJetsAuxDyn", "AnalysisElectronsAuxDyn", "AnalysisMuonsAuxDyn", "AnalysisTauJetsAuxDyn", "AnalysisPhotonsAuxDyn"]]):
    #     continue

    try:
        t.arrays([key])
        print("read branch", key)
        new_size_in_MB = t.file.source.num_requested_bytes/1000**2
        size_per_key_in_MB[key] = new_size_in_MB - cur_size_in_MB
        cur_size_in_MB = new_size_in_MB
    except:
        print("cannot read branch", key)
    
size_per_key_in_MB

read branch AnalysisJetsAuxDyn.pt
read branch AnalysisJetsAuxDyn.eta
read branch AnalysisJetsAuxDyn.phi
read branch AnalysisJetsAuxDyn.m
read branch AnalysisJetsAuxDyn.JetConstitScaleMomentum_pt
read branch AnalysisJetsAuxDyn.JetConstitScaleMomentum_eta
read branch AnalysisJetsAuxDyn.JetConstitScaleMomentum_phi
read branch AnalysisJetsAuxDyn.JetConstitScaleMomentum_m
read branch AnalysisJetsAuxDyn.btaggingLink
read branch AnalysisJetsAuxDyn.btaggingLink/AnalysisJetsAuxDyn.btaggingLink.m_persKey
read branch AnalysisJetsAuxDyn.btaggingLink/AnalysisJetsAuxDyn.btaggingLink.m_persIndex
read branch AnalysisJetsAuxDyn.ActiveArea4vec_eta
read branch AnalysisJetsAuxDyn.ActiveArea4vec_m
read branch AnalysisJetsAuxDyn.ActiveArea4vec_phi
read branch AnalysisJetsAuxDyn.ActiveArea4vec_pt
read branch AnalysisJetsAuxDyn.DetectorEta
read branch AnalysisJetsAuxDyn.EMFrac
read branch AnalysisJetsAuxDyn.JVFCorr
read branch AnalysisJetsAuxDyn.Timing
read branch AnalysisJetsAuxDyn.NumTrkPt1000
read branch A

{'AnalysisJetsAuxDyn.pt': 1.2115909999999999,
 'AnalysisJetsAuxDyn.eta': 1.2643980000000004,
 'AnalysisJetsAuxDyn.phi': 1.2587349999999997,
 'AnalysisJetsAuxDyn.m': 1.194972,
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_pt': 1.2220149999999999,
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_eta': 1.2667650000000004,
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_phi': 1.260966999999999,
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_m': 1.2342960000000005,
 'AnalysisJetsAuxDyn.btaggingLink': 0.3213790000000003,
 'AnalysisJetsAuxDyn.btaggingLink/AnalysisJetsAuxDyn.btaggingLink.m_persKey': 0.09002199999999938,
 'AnalysisJetsAuxDyn.btaggingLink/AnalysisJetsAuxDyn.btaggingLink.m_persIndex': 0.1365690000000015,
 'AnalysisJetsAuxDyn.ActiveArea4vec_eta': 1.2656609999999997,
 'AnalysisJetsAuxDyn.ActiveArea4vec_m': 1.1722640000000002,
 'AnalysisJetsAuxDyn.ActiveArea4vec_phi': 1.2603269999999984,
 'AnalysisJetsAuxDyn.ActiveArea4vec_pt': 1.1586070000000017,
 'AnalysisJetsAuxDyn.DetectorEta': 1.26527199

In [8]:
print(f"all branches: {sum(size_per_key_in_MB.values())} MB")  # doesn't include initial metadata read

all branches: 554.5054250000001 MB


In [10]:
import os
file_size = os.path.getsize(fname) / 1000**2
file_size

554.994903

In [17]:
t.file.source.num_requested_bytes/1000**2

1502.043129

sanity check: looks like we read pretty much everything, seems good

In [11]:
import json

size_per_key_in_MB["_file_size"] = file_size

with open("branch_sizes.json", "w") as f:
    f.write(json.dumps(size_per_key_in_MB, sort_keys=True, indent=4))

## sort branches, find branches to hit 10% read

In [47]:
import pathlib

size_per_key_in_MB = json.loads(pathlib.Path("branch_sizes.json").read_text())
file_size = size_per_key_in_MB["_file_size"]

list_of_branches_to_read = []
size_of_branches = 0

# veto the metadata branch and also possibly problematic ones that could vary a lot across files
branches_to_veto = ["_file_size", "AnalysisJetsAuxDyn.GhostTrack", "AnalysisJetsAuxDyn.btaggingLink"] #, "HLTNav_Summary_DAODSlimmedAuxDyn.decisions", "InDetTrackParticlesAuxDyn.definingParametersCovMatrixOffDiag"]

for k, v in reversed(sorted(size_per_key_in_MB.items(), key=lambda it: it[1])):
    # only keep branches relating to specific physics objects
    if not any([vv in k for vv in ["AnalysisJetsAuxDyn", "AnalysisElectronsAuxDyn", "AnalysisMuonsAuxDyn"]]):
        continue
    
    if k in branches_to_veto:
        continue

    list_of_branches_to_read.append(k)
    size_of_branches += v
    if size_of_branches / file_size > 0.10:  # target: 25%
        break

for b in list_of_branches_to_read:
    print(f"{b}: {size_per_key_in_MB[b] / file_size:.2%}")

AnalysisJetsAuxDyn.EnergyPerSampling: 1.56%
AnalysisJetsAuxDyn.SumPtTrkPt500: 1.49%
AnalysisJetsAuxDyn.TrackWidthPt1000: 0.83%
AnalysisJetsAuxDyn.NumTrkPt500: 0.49%
AnalysisJetsAuxDyn.NumTrkPt1000: 0.36%
AnalysisJetsAuxDyn.SumPtChargedPFOPt500: 0.30%
AnalysisJetsAuxDyn.Timing: 0.23%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_eta: 0.23%
AnalysisJetsAuxDyn.ActiveArea4vec_eta: 0.23%
AnalysisJetsAuxDyn.DetectorEta: 0.23%
AnalysisJetsAuxDyn.eta: 0.23%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_phi: 0.23%
AnalysisJetsAuxDyn.ActiveArea4vec_phi: 0.23%
AnalysisJetsAuxDyn.phi: 0.23%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_m: 0.22%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_pt: 0.22%
AnalysisJetsAuxDyn.Width: 0.22%
AnalysisJetsAuxDyn.EMFrac: 0.22%
AnalysisJetsAuxDyn.pt: 0.22%
AnalysisJetsAuxDyn.m: 0.22%
AnalysisJetsAuxDyn.ActiveArea4vec_m: 0.21%
AnalysisJetsAuxDyn.ActiveArea4vec_pt: 0.21%
AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksWidth: 0.19%
AnalysisJetsAuxDyn.JVFCorr: 0.17%
AnalysisJets

In [48]:
list_of_branches_to_read

['AnalysisJetsAuxDyn.EnergyPerSampling',
 'AnalysisJetsAuxDyn.SumPtTrkPt500',
 'AnalysisJetsAuxDyn.TrackWidthPt1000',
 'AnalysisJetsAuxDyn.NumTrkPt500',
 'AnalysisJetsAuxDyn.NumTrkPt1000',
 'AnalysisJetsAuxDyn.SumPtChargedPFOPt500',
 'AnalysisJetsAuxDyn.Timing',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_eta',
 'AnalysisJetsAuxDyn.ActiveArea4vec_eta',
 'AnalysisJetsAuxDyn.DetectorEta',
 'AnalysisJetsAuxDyn.eta',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_phi',
 'AnalysisJetsAuxDyn.ActiveArea4vec_phi',
 'AnalysisJetsAuxDyn.phi',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_m',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_pt',
 'AnalysisJetsAuxDyn.Width',
 'AnalysisJetsAuxDyn.EMFrac',
 'AnalysisJetsAuxDyn.pt',
 'AnalysisJetsAuxDyn.m',
 'AnalysisJetsAuxDyn.ActiveArea4vec_m',
 'AnalysisJetsAuxDyn.ActiveArea4vec_pt',
 'AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksWidth',
 'AnalysisJetsAuxDyn.JVFCorr',
 'AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksC1',
 'AnalysisJetsAuxDyn.PSFrac',


## sanity check: load file again, read these branches, ensure we hit the right fraction of the file

In [51]:
t = uproot.open({fname: "CollectionTree"})

cur_size_in_MB = t.file.source.num_requested_bytes/1000**2  # non-zero at the start (some metadata read)

t.arrays(list_of_branches_to_read)

new_size_in_MB = t.file.source.num_requested_bytes/1000**2
print(cur_size_in_MB, new_size_in_MB)

0.831569 54.268776


In [52]:
(new_size_in_MB - cur_size_in_MB) / file_size

0.09628414010858041